# Day-22

## Part 2

`x` — card position (0-based)
`D` — deck size

### Movements

#### deal into new stack
$$f(x) = (D - x - 1) \mod D = (-x - 1) \mod D$$

#### cut $N$
$$g(x, N) = (x - N) \mod D$$

#### deal with increment $M$
$$h(x, M) = xM \mod D$$

### Observations

1. $f(f(x)) = f(x)$
1. $f(x) = g(h(x, -1), 1)$
1. $g(g(x, N_1), N_2 = g((x - N_1) \mod D, N_2) = (((x - N_1) \mod D) - N_2) \mod D = (x - N_1 - N_2) \mod D = g(x, (N_1 + N_2) \mod D)$
1. $h(h(x, M_1), M_2) = h(xM_1 \mod D, M_2) = (xM_1 \mod D) \cdot M_2 \mod D = x \cdot (M_1 \cdot M_2 \mod D) \mod D = h(x, M_1 \cdot M_2 \mod D)$
1. $g(h(x, M), N) = g(xM \mod D, N) = (xM \mod D) - N \mod D = (xM - N) \mod D$
1. $h(g(x, N), M) = h((x - N) \mod D, M) = ((x - N) \mod D) \cdot M \mod D = (xM - NM) \mod D = g(h(x, M), NM)$

### Key thought

looks like the whole shuffle is just $F(x) = (xM + N) \mod D$!

### Replacements

based on observation number

* `1` => (deal into new stack, deal into new stack) => $\emptyset$
* `2` => deal into new stack -> deal with increment $-1$
* `3` => (cut $N_1$, cut $N_2$) -> cut $(N_1 + N_2) \mod D$ 
* `4` => (deal with increment $M_1$, deal_with_increment $M_2$) -> deal with increment $M_1 \cdot M_2 \mod D$
* `6` => (cut $N$, deal with increment $M$) -> (deal with increment $M$, cut $N \cdot M \mod D$)

Let's check the [key thought](#Key-thought)!

In [57]:
#[derive(Debug)]
/// xM + N
struct Model {
    n: i64,
    m: i64,
    deck: i64,
}

impl Model {
    fn new(deck: i64) -> Self {
        Self {
            n: 0,
            m: 1,
            deck,
        }
    }
    
    fn cut(&mut self, n: i64) {
        self.n = (self.n - n) % self.deck;
    }
    
    fn deal_in(&mut self) {
        self.deal_inc(-1);
        self.cut(1);
    }
    
    fn deal_inc(&mut self, inc: i64) {
        self.m = (self.m * inc) % self.deck;
        self.n = (self.n * inc) % self.deck;
    }
    
    fn eval(&self, x: i64) -> i64 {
        (x as i128 * self.m as i128 + self.n as i128).rem_euclid(self.deck as i128) as i64
    }
    
    fn shuffle(&self) -> Vec<i64> {
        let mut with_order = (0..self.deck)
            .map(|n| (n, self.eval(n)))
            .collect::<Vec<_>>();
        
        with_order.sort_unstable_by_key(|(_n, order)| *order);
        
        with_order.into_iter().map(|(n, _order)| n).collect()
    }
}

In [12]:
let mut m = Model::new(10);
m.deal_in();
m.deal_in();

m.shuffle()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [47]:
// deal with increment 7
// deal with increment 9
// cut -2
// Result: 6 3 0 7 4 1 8 5 2 9

let mut m = Model::new(10);
m.deal_inc(7);
m.deal_inc(9);
m.cut(-2);

m.shuffle()

[6, 3, 0, 7, 4, 1, 8, 5, 2, 9]

In [52]:
// cut 6
// deal with increment 7
// deal into new stack
// Result: 3 0 7 4 1 8 5 2 9 6

let mut m = Model::new(10);
m.cut(6);
m.deal_inc(7);
m.deal_in();

m.shuffle()

[3, 0, 7, 4, 1, 8, 5, 2, 9, 6]

In [49]:
// deal into new stack
// cut -2
// deal with increment 7
// cut 8
// cut -4
// deal with increment 7
// cut 3
// deal with increment 9
// deal with increment 3
// cut -1
// Result: 9 2 5 8 1 4 7 0 3 6

let mut m = Model::new(10);
m.deal_in();
m.cut(-2);
m.deal_inc(7);
m.cut(8);
m.cut(-4);
m.deal_inc(7);
m.cut(3);
m.deal_inc(9);
m.deal_inc(3);
m.cut(-1);

m.shuffle()

[9, 2, 5, 8, 1, 4, 7, 0, 3, 6]

Yaay!!!

### What's left

What does it mean to apply model N times? (all equalities are $\pmod D$)

* $F(F(x)) = F(xM + N) = (((xM + N)) \cdot M + N)$
* => $F(F(x)) = (xM + N) * M + N)$
* => $F(F(x)) = xM^2 + M \cdot (N + 1)$
* ...
* => F(...) recursively applied $n$ times = $x \cdot M^n + N \cdot M^{n-1} + N \cdot M^{n-2} + ... + N$  
  $= x \cdot M^n + N \cdot (M^0 + M^1 + M^2 + ... + M^{n - 1})$  
  $= x \cdot M^n + N \cdot (\frac{M^{n} - 1}{M - 1})$

#### Interesting problems

While computing sum of powers of $M$ we can't just do $\frac{M^{n} - 1}{M - 1}$ as we're in the modular arithmetics.

After spending some time grieving, I understood the we just have to find $(M - 1)^{-1} \pmod{D}$ — an inverse modular root for $M - 1$ in field $D$, and then do $(M^{n} - 1) \cdot (M - 1)^{-1} \pmod{D}$ (spoiler: this "inverse modular root" approach is going to be of use one more time later). 

In [152]:
let model_in_question = Model { n: 43440179824106, m: -56568781986389, deck: 119315717514047 };

#[inline(always)]
fn mult128(x: i64, y: i64, modulo: i64) -> i64 { 
    ((x as i128 * y as i128).rem_euclid(modulo as i128)) as i64
}

fn sum_of_pows_naive(x: i64, pow_to: usize, modulo: i64) -> i64 {
    let mut cur = 1;
    let mut s: i64 = 0;

    for _ in 0..=pow_to {
        s = (s + cur).rem_euclid(modulo);
        cur = mult128(cur, x, modulo);
    }

    s
}

fn bin_pow(mut x: i64, mut pow_to: usize, modulo: i64) -> i64 {
    let mut res = 1;
    while pow_to > 0 {
        if pow_to & 1 == 1 {
            res = mult128(res, x, modulo);
        }
        x = mult128(x, x, modulo);
        pow_to >>= 1;
    }
    res
}

struct GcdSolution {
    gcd: i64,
    x: i64,
    y: i64,
}

/// function extended_gcd(a, b)
//     (old_r, r) := (a, b)
//     (old_s, s) := (1, 0)
//     (old_t, t) := (0, 1)
    
//     while r ≠ 0 do
//         quotient := old_r div r
//         (old_r, r) := (r, old_r − quotient × r)
//         (old_s, s) := (s, old_s − quotient × s)
//         (old_t, t) := (t, old_t − quotient × t)
    
//     output "Bézout coefficients:", (old_s, old_t)
//     output "greatest common divisor:", old_r
//     output "quotients by the gcd:", (t, s)
fn gcd_extended(a: i64, b: i64) -> GcdSolution {
    let mut old_r = a;
    let mut r = b;
    
    let mut old_s = 1;
    let mut s = 0;
    
    let mut old_t = 0;
    let mut t = 1;
    
    while r != 0 {
        let quot = old_r / r;
        
        let r_temp = r;
        r = old_r - quot * r;
        old_r = r_temp;
        
        let s_temp = s;
        s = old_s - quot * s;
        old_s = s_temp;
        
        let t_temp = t;
        t = old_t - quot * t;
        old_t = t_temp;
    }
    
    GcdSolution {
        x: old_s,
        y: old_t,
        gcd: old_r,
    }
}

fn inverse_mod_root(n: i64, modulo: i64) -> i64 {
    let gcd_extended = gcd_extended(n, modulo);
    let gcd = gcd_extended.gcd;
    assert_eq!(gcd.abs(), 1, "numbers must be coprime for this method to work!");
    
    let x = gcd_extended.x * gcd;
    (x.rem_euclid(modulo) + modulo).rem_euclid(modulo)
}

/// "a" for analytics
fn sum_of_pows_a(x: i64, pow_to: usize, modulo: i64) -> i64 {
    let root = inverse_mod_root(x - 1, modulo);
    
    mult128(bin_pow(x, pow_to + 1, modulo) - 1, root, modulo)
}

assert_eq!(
    sum_of_pows(-3, 1, model_in_question.deck),
    sum_of_pows_a(-3, 1, model_in_question.deck),
);

assert_eq!(
    sum_of_pows(model_in_question.m, 30, model_in_question.deck),
    sum_of_pows_a(model_in_question.m, 30, model_in_question.deck),
);

sum_of_pows_a(model_in_question.m, 101_741_582_076_660, model_in_question.deck)

116122951247704

### Part 2 Mystery, or I Am an Idiot :(

In part two you need not to find, where is the card #n, but **what card is it on place #n**...

So we have our model ($F(x) = x * M + N$) and we need to solve it for known (found) $M$ and $N$ and $n$ set to 2020.

* $F(x) = x \cdot M + N$
* => $x \cdot M + N = n$
* => $x \cdot M = n - N$
* => $x = (n - N) \cdot M^{-1}$, where $M^{-1}$ is inverse modular root for $M$ in field modulo $D$.